# 🛒 E-Commerce Sales Data Analysis Pipeline
## Data Exploration & Quality Assessment

---

### 📋 Project Overview
This notebook explores raw e-commerce data from the Brazilian marketplace Olist (2016-2018). 
We analyze **100K+ orders** across multiple datasets to identify data quality issues before 
building our ETL pipeline.

---

### 🎯 Objectives
1. **Load** real-world e-commerce data from CSV files
2. **Explore** data structure, types, and relationships
3. **Identify** data quality issues (missing values, wrong types, duplicates)
4. **Document** findings for the data cleaning phase

---

### 📊 Datasets Used

| Dataset | Records | Description |
|---------|---------|-------------|
| **Customers** | ~99K | Customer information (ID, location) |
| **Orders** | ~99K | Order details (status, timestamps) |
| **Order Items** | ~112K | Products in each order |
| **Products** | ~32K | Product catalog (names, categories, prices) |
| **Sellers** | ~3K | Seller information |
| **Payments** | ~103K | Payment transactions |
| **Reviews** | ~100K | Customer ratings and comments |

---

### 🔧 Tools & Libraries
- **pandas**: Data manipulation and analysis
- **Python**: Data exploration and scripting
- **PostgreSQL**: Target database 
- **Tableau**: Visualization

---

### 📂 Data Source
**Kaggle:** [Brazilian E-Commerce Public Dataset by Olist](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce)

**Files located in:** `../data/raw/`

---

### 🚀 What We'll Discover

#### Expected Data Quality Issues:
- ❌ Date columns stored as text (need datetime conversion)
- ❌ Missing values in various fields
- ❌ Potential duplicates
- ❌ Product names in Portuguese (may need translation)
- ❌ Inconsistent formatting

#### Key Questions:
- How much data is missing?
- Are dates in the correct format?
- Do we have duplicates?
- What's the date range of our data?
- How many unique customers vs orders?

---

### 📝 Next Steps (After This Notebook)
1. **Data Cleaning** (`clean_data.py`) - Fix identified issues with pandas
2. **Database Design** - Create star schema for analytics
3. **ETL Pipeline** - Load cleaned data to PostgreSQL
4. **SQL Analysis** - Business metrics and customer insights
5. **Visualization** - Tableau dashboard

---

**Let's begin the exploration! 👇**

In [12]:
import pandas as pd 
import os

In [3]:
# to show all columns when displaying data
pd.set_option('display.max_columns', None)

# displays up to 100 rows instead of default 10
pd.set_option('display.max_rows', 100)

In [4]:
# File path
data_dir = '../data/raw/'

print("Available files: ")
for file in os.listdir(data_dir):
    if file.endswith('.csv'):
        size = os.path.getsize(os.path.join(data_dir, file)) / 1024 / 1024 # bytes->kilobytes->megabytes
        print(f" {file}: {size: .2f} MB")

Available files: 
 olist_sellers_dataset.csv:  0.17 MB
 product_category_name_translation.csv:  0.00 MB
 olist_orders_dataset.csv:  16.84 MB
 olist_order_items_dataset.csv:  14.72 MB
 olist_customers_dataset.csv:  8.62 MB
 olist_geolocation_dataset.csv:  58.44 MB
 olist_order_payments_dataset.csv:  5.51 MB
 olist_order_reviews_dataset.csv:  13.78 MB
 olist_products_dataset.csv:  2.27 MB


In [5]:
# Load main datasets
print("Loading datasets...")

customers = pd.read_csv(data_dir + 'olist_customers_dataset.csv')
orders = pd.read_csv(data_dir + 'olist_orders_dataset.csv')
order_items = pd.read_csv(data_dir + 'olist_order_items_dataset.csv')
products = pd.read_csv(data_dir + 'olist_products_dataset.csv')

print("✅ Loaded!")
print(f"Customers: {len(customers):,} rows") #:, - formats the number with commas like 1,000
print(f"Orders: {len(orders):,} rows")
print(f"Order Items: {len(order_items):,} rows")
print(f"Products: {len(products):,} rows")

Loading datasets...
✅ Loaded!
Customers: 99,441 rows
Orders: 99,441 rows
Order Items: 112,650 rows
Products: 32,951 rows


In [6]:
print("\n CUSTOMERS DATA")
print("="*50)

print("\nFirst 5 rows: ")
print(customers.head())

print("\nColumn names: ")
print(customers.columns.tolist())

print("\nData Types: ")
print(customers.dtypes)

print("\nDataFrame shape (rows, columns): ")
print(customers.shape)

print("\nMissing values: ")
print(customers.isnull().sum())



 CUSTOMERS DATA

First 5 rows: 
                        customer_id                customer_unique_id  \
0  06b8999e2fba1a1fbc88172c00ba8bc7  861eff4711a542e4b93843c6dd7febb0   
1  18955e83d337fd6b2def6b18a428ac77  290c77bc529b7ac935b93aa66c333dc3   
2  4e7b3e00288586ebd08712fdd0374a03  060e732b5b29e8181a18229c7b0b2b5e   
3  b2b6027bc5c5109e529d4dc6358b12c3  259dac757896d24d7702b9acbbff3f3c   
4  4f2d8ab171c80ec8364f7c12e35b23ad  345ecd01c38d18a9036ed96c73b8d066   

   customer_zip_code_prefix          customer_city customer_state  
0                     14409                 franca             SP  
1                      9790  sao bernardo do campo             SP  
2                      1151              sao paulo             SP  
3                      8775        mogi das cruzes             SP  
4                     13056               campinas             SP  

Column names: 
['customer_id', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']

Da

In [7]:
print("\nORDERS DATA")
print("="*50)

print("\nFirst 5 rows: ")
print(orders.head())

print("\nColumns: ")
print(orders.columns.tolist())

print("\nData types: ")
print(orders.dtypes)

print("\nMissing values: ")
missing = orders.isnull().sum()
print(missing[missing > 0]) # show columns with nulls only

print("\nOrder statuses: ")
print(orders['order_status'].value_counts())

print("\nDate range: ")
print(f"From: {orders['order_purchase_timestamp'].min()}")
print(f"To: {orders['order_purchase_timestamp'].max()}")


ORDERS DATA

First 5 rows: 
                           order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
2  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   
3  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
4  ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c   

  order_status order_purchase_timestamp    order_approved_at  \
0    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
1    delivered      2018-07-24 20:41:37  2018-07-26 03:24:27   
2    delivered      2018-08-08 08:38:49  2018-08-08 08:55:23   
3    delivered      2017-11-18 19:28:06  2017-11-18 19:45:59   
4    delivered      2018-02-13 21:18:39  2018-02-13 22:20:29   

  order_delivered_carrier_date order_delivered_customer_date  \
0          2017-10-04 19:55:00           2017-10-10 21:25:13   
1          2018-07-

In [9]:
print("\nDUPLICATES CHECK")
print("="*50)

print(f"Duplicate customers: {customers.duplicated().sum()}")
print(f"Duplicated customer IDs: {customers['customer_id'].duplicated().sum()}")

print(f"\nDuplicate orders: {orders.duplicated().sum()}")
print(f"Duplicate order IDs: {orders['order_id'].duplicated().sum()}")

print(f"\nDuplicate order items: {order_items.duplicated().sum()}")

# Check if same customer ordered multiple times (expected)
print(f"\nCustomers with multiple orders:")
customer_order_counts = orders['customer_id'].value_counts()
print(f"  Total unique customers: {len(customer_order_counts)}")
print(f"  Customers with 2+ orders: {(customer_order_counts > 1).sum()}")
print(f"  Max orders by one customer: {customer_order_counts.max()}")


DUPLICATES CHECK
Duplicate customers: 0
Duplicated customer IDs: 0

Duplicate orders: 0
Duplicate order IDs: 0

Duplicate order items: 0

Customers with multiple orders:
  Total unique customers: 99441
  Customers with 2+ orders: 0
  Max orders by one customer: 1


In [11]:
print("\nDATA QUALITY ISSUES TO FIX: ")
print("="*50)

issues = []

# Check date columns
date_cols = [col for col in orders.columns if 'timestamp' in col or 'date' in col]
if any(orders[col].dtype == 'object' for col in date_cols):
    issues.append("Date columns stored as text(need conversion)")

# Check orders for nulls
if orders.isnull().sum().sum() > 0:
    issues.append(f"Missing values in orders: {orders.isnull().sum().sum()} cells")

# Check products for nulls
if products.isnull().sum().sum() > 0:
    issues.append(f"Missing values in products: {products.isnull().sum().sum()} cells")

# Display issues
for i, issue in enumerate(issues, 1):
    print(f"{i}. {issue}")

print(f"\n✅ Total issues to address: {len(issues)}")
print("\n📝 Next step: Clean these issues with pandas!")


DATA QUALITY ISSUES TO FIX: 
1. Date columns stored as text(need conversion)
2. Missing values in orders: 4908 cells
3. Missing values in products: 2448 cells

✅ Total issues to address: 3

📝 Next step: Clean these issues with pandas!
